In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image

# --- 1. SETUP PATHS AND PARAMETERS ---
# Path updated as per your request.
DATA_DIR = r"C:\Users\adity\Downloads\dataset\Dataset"
IMAGE_SIZE = (160, 160)
BATCH_SIZE = 32

# --- 2. DEFINITIVE DATA CLEANUP & STANDARDIZATION ---
print("\nStarting definitive data cleanup & standardization...")
num_processed = 0
for folder_name in ("Cat", "Dog"): 
    folder_path = os.path.join(DATA_DIR, folder_name)
    if not os.path.isdir(folder_path):
        print(f"Error: Directory not found. Please ensure the path is correct and contains 'Cat' and 'Dog' subfolders.")
        print(f"Path checked: {folder_path}")
        exit()
    for fname in os.listdir(folder_path):
        num_processed += 1
        if num_processed % 500 == 0:
            print(f"   Processing image {num_processed}...")
        
        fpath = os.path.join(folder_path, fname)
        try:
            with Image.open(fpath) as img:
                img.load()
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                img.save(fpath, 'JPEG')
        except Exception as e:
            print(f"DELETING corrupted file: {fpath} (Reason: {e})")
            try:
                os.remove(fpath)
            except OSError as oe:
                print(f"   Error removing file: {oe}")
print(f"Cleanup complete. Processed and standardized {num_processed} images.")


# --- 3. CREATE THE DATA PIPELINE ---
print("\nCreating data pipelines...")
train_dataset, validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_names=['Cat', 'Dog'] 
)

# --- 4. CREATE DATA AUGMENTATION & CONFIGURE FOR PERFORMANCE ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
])

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# --- 5. BUILD AND COMPILE THE MODEL ---
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,), include_top=False, weights='imagenet'
)
base_model.trainable = False

inputs = layers.Input(shape=(160, 160, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# --- 6. TRAIN THE MODEL ---
print("\n🚀 Starting model training...")
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset
)

# --- 7. SAVE THE FINAL MODEL ---
model.save('cat_dog_classifier_local.keras')
print("\n✅ Model saved successfully!")



Starting definitive data cleanup & standardization...
   Processing image 500...
   Processing image 1000...
   Processing image 1500...
   Processing image 2000...
   Processing image 2500...
   Processing image 3000...
   Processing image 3500...
   Processing image 4000...
   Processing image 4500...
   Processing image 5000...
   Processing image 5500...
   Processing image 6000...
   Processing image 6500...
   Processing image 7000...
   Processing image 7500...
   Processing image 8000...
   Processing image 8500...
   Processing image 9000...
   Processing image 9500...
   Processing image 10000...
   Processing image 10500...
   Processing image 11000...
   Processing image 11500...
   Processing image 12000...
   Processing image 12500...
   Processing image 13000...
   Processing image 13500...
   Processing image 14000...
   Processing image 14500...
   Processing image 15000...
   Processing image 15500...
   Processing image 16000...
   Processing image 16500...
   Proc